<a href="https://colab.research.google.com/github/czhurdlespeed/TFDevCert/blob/main/TFCert_Practice/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-52c01569-fd78-9fd4-66b9-907293778d36)


## Import Helper Functions

In [2]:
!wget https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-02-15 00:37:21--  https://github.com/mrdbourke/tensorflow-deep-learning/raw/main/extras/helper_functions.py
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py [following]
--2024-02-15 00:37:21--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-02-15 00:37:21 (71.1 MB/s) - ‘helper_

## Get a text dataset

- Using Kaggle's Introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# Uzip data
unzip_data("nlp_getting_started.zip")

--2024-02-15 00:37:26--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   711KB/s    in 0.8s    

2024-02-15 00:37:27 (711 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [6]:
# Shuffle training datafram
train_df_shuffled = train_df.sample(frac=1)
train_df_shuffled.head()

,id,keyword,location,text,target
1920,2761,curfew,NaN,for some reason im listening to curfew overtim...,0
4814,6852,mass%20murder,"Magnolia, Fiore",@ColdMpress You up to commiting mass murder to...,1
1420,2049,casualties,"Afghanistan, USA",#Afghanistan: sharp rise in women and children...,1
5350,7638,pandemonium,NaN,- Pandemonium In Aba As Woman Delivers Baby W...,0
4300,6106,hellfire,Right next to Compton,@IAN_Hellfire I got it for the mistake but bos...,0


In [7]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class are there?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many locations?
train_df.location.value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [10]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", r"(real disaster)" if target > 0 else r"(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
A look at state actions a year after FergusonÛªs upheaval  http://t.co/G5ZsRU0zVQ

---

Target: 1 (real disaster)
Text:
Strict liability in the context of an airplane accident: Pilot error is a common component of most aviation cr... http://t.co/6CZ3bOhRd4

---

Target: 0 (not real disaster)
Text:
@stacedemon oh shit!

---

Target: 0 (not real disaster)
Text:
The hurricane mixxtail kinda tastes like the watermelon four loko. ???????? @brittsand9

---

Target: 0 (not real disaster)
Text:
@amaramin3 Meek is definitely capable of body bagging his ass on the track Drake was just smooth as fuck with it!

---



In [12]:
print("Hello\'\\ \n")

Hello'\ 



In [13]:
print(r"Hello\'\\ \n")

Hello\'\\ \n


### Split data into training and validation sets

In [14]:
from sklearn.model_selection import train_test_split
train_sentences, validation_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                                   train_df_shuffled["target"].to_numpy(),
                                                                                   test_size=0.1)


In [15]:
# Check the lengths
len(train_sentences), len(train_labels), len(validation_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(["Winter Desolation of Death is also on Tumblr:  http://t.co/93DM6gnWwC  Al Necro's reviews interviews &amp; more!",
        'SCREAMING IN 22 DIFFERENT LANGUAGES http://t.co/rDfaAKKbNJ',
        'Monsoon Flooding Kills Dozens In Myanmar Prompting Calls For Help - http://t.co/r7vPaKlhvI',
        'My dog\x89Ûªs just blown his kennel up \x89ÛÒ Bloody Yorkshire Terrorist',
        'RSS: Russia begins mass destruction of illegally imported food   http://t.co/r6JDj9kIGm',
        "WRAPUP 2-U.S. cable TV companies' shares crushed after Disney disappoints http://t.co/wWFACu6NFt",
        'Here\x89Ûªs how media in Pakistan covered the capture of terrorist Mohammed Naved http://t.co/f7WqpCEkg2',
        'E-Hutch is da bomb ?? http://t.co/aqmpxzo3V1',
        "i could die by falling in a sinkhole and i'd still be blamed for it",
        '@KirCut1 lets get a dope picture together and have the dopest explosion ????'],
       dtype=object),
 array([0, 0, 1, 0, 1, 0, 0, 0, 0, 0]))

## Converting text into numbers

In [17]:
# Embedding - every word gets turned into a vector; you can specify the dimensions of the vector
  # gets updated as the network learns; therfore it can learn relationships between tokens
  # Pretrained embedding libraries e.g. word2vec, GloVe, etc.


In [18]:
# Text vectorization (tokenization)
train_sentences[:5]

array(["Winter Desolation of Death is also on Tumblr:  http://t.co/93DM6gnWwC  Al Necro's reviews interviews &amp; more!",
       'SCREAMING IN 22 DIFFERENT LANGUAGES http://t.co/rDfaAKKbNJ',
       'Monsoon Flooding Kills Dozens In Myanmar Prompting Calls For Help - http://t.co/r7vPaKlhvI',
       'My dog\x89Ûªs just blown his kennel up \x89ÛÒ Bloody Yorkshire Terrorist',
       'RSS: Russia begins mass destruction of illegally imported food   http://t.co/r6JDj9kIGm'],
      dtype=object)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# use the default textvectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in our vocabulary; no cap on size of vocab
                                    standardize='lower_and_strip_punctuation',
                                    split='whitespace',
                                    ngrams=None, # create groups of n-words?
                                    output_mode='int', # how to map tokens to numbers
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [20]:
# find the average number of tokens in the training tweets
round((sum([len(i.split()) for i in train_sentences]))/len(train_sentences))

15

In [21]:
# Setup text vectorization variables
max_vocab_length = 10000 # max num of words to have in vocab
max_length = 15 # max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [22]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [23]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[279,   3, 198,   4,  13, 633,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence} \
      \n\nVectorized versions:")
text_vectorizer([random_sentence])

Original text:
Here I'm the UK there isn't a deluge of Canadian themed tops around...The timing was perfect. I can't quite believe it. Mad.       

Vectorized versions:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 136,   35,    2, 1164,   72,  730,    3,  353,    6,    1, 7620,
        7431,    1, 7518,   24]])>

In [25]:
# Get the unique words in the vocab
words_in_vocab = text_vectorizer.get_vocabulary() # Get all the unique words in our training data
# Most common words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']


### Creating an Embedding using an Embedding Layer

In [26]:
# taken tokenized text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128, # best to choose number divisible by 8
                             input_length=max_length # how long is each input
                             )
embedding

In [27]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
\n\n Embedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @tsunami_esh ESH PLEASE OKAY!

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00702808, -0.01750171,  0.01600697, ...,  0.00656454,
          0.00893974,  0.0334732 ],
        [ 0.01195489, -0.03780044, -0.02675246, ..., -0.04091066,
          0.00488741,  0.04452934],
        [ 0.01314821,  0.04535014,  0.00993141, ..., -0.01157696,
         -0.0148586 , -0.04603958],
        ...,
        [-0.02745006,  0.02600152,  0.02698651, ...,  0.02776719,
          0.00094011, -0.0284946 ],
        [-0.02745006,  0.02600152,  0.02698651, ...,  0.02776719,
          0.00094011, -0.0284946 ],
        [-0.02745006,  0.02600152,  0.02698651, ...,  0.02776719,
          0.00094011, -0.0284946 ]]], dtype=float32)>

In [28]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00702808, -0.01750171,  0.01600697, -0.03162352, -0.03402396,
         0.01088871,  0.01781398, -0.01165354,  0.01676314, -0.04207031,
         0.04683695, -0.03769189,  0.02330269,  0.04744413, -0.01064082,
         0.01025004,  0.04803946,  0.00926092, -0.03362598,  0.0472951 ,
        -0.03155857, -0.03738863, -0.02544698, -0.04200631, -0.03716014,
        -0.004803  , -0.02364744, -0.0471893 , -0.01706095, -0.04702773,
         0.02870512, -0.02922933, -0.02926018,  0.00551163, -0.02145708,
         0.03710195, -0.01924927,  0.01608566, -0.02005054, -0.03777642,
        -0.02453195, -0.04047524, -0.01131859, -0.00044613, -0.00562936,
        -0.04232783, -0.01675671, -0.01950996, -0.00781028, -0.01794662,
         0.02538193, -0.04154316, -0.00311259,  0.01897924,  0.01888591,
        -0.04376629, -0.00671715,  0.00358726, -0.00856633,  0.0249292 ,
        -0.0238906 , -0.04066932,  0.03142761, -0.02105639, -0.02009746,
  

## Modeling a text dataset
* Model 0: Naive Bayes (baseline) [sklearn ML map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional NN (CNN)
* Model 6: TF Hub Pretrained Feature Extractor (transfer learning)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer # turn text into numbers/vectors
from sklearn.pipeline import Pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB()) # clf == "classifier"
])
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
# Evaluate our baseline model
baseline_score = model_0.score(validation_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.79%


In [31]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [32]:
# Make predictions
baseline_preds = model_0.predict(validation_sentences)
baseline_preds[:20]

array([0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1])

### Creating an evaluation function for our model experiments

Create on to compare our model's predictions with the truth labels
* Accuracy
* Precision
* Recall
* F1-score

In [34]:
# Function to evaluate: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall, and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [36]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.79002624671917,
 'precision': 0.805269198642199,
 'recall': 0.7979002624671916,
 'f1': 0.7929822281460398}

# Model 1: Feed-forward neural network (dense model)

In [57]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save Tensorboard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1D strings
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # Create output layer, want binary outputs
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [58]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [59]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [60]:
# Fit model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(validation_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240215-011030
Epoch 1/5
215/215 [==============================] - 8s 34ms/step - loss: 0.5846 - accuracy: 0.7542 - val_loss: 0.5282 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.4200 - accuracy: 0.8371 - val_loss: 0.4525 - val_accuracy: 0.8018
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3328 - accuracy: 0.8708 - val_loss: 0.4363 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2756 - accuracy: 0.8964 - val_loss: 0.4359 - val_accuracy: 0.8097
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2332 - accuracy: 0.9169 - val_loss: 0.4534 - val_accuracy: 0.8058


In [61]:
# Check the results
model_1.evaluate(validation_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4534 - accuracy: 0.8058


[0.4533763527870178, 0.8057742714881897]

In [62]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(validation_sentences)
model_1_pred_probs

24/24 [==============================] - 0s 2ms/step


array([[0.21735752],
       [0.5040701 ],
       [0.8186321 ],
       [0.569228  ],
       [0.29355854],
       [0.97278124],
       [0.19997315],
       [0.059449  ],
       [0.0600296 ],
       [0.28930956],
       [0.73786026],
       [0.82243884],
       [0.10227519],
       [0.4170531 ],
       [0.6401937 ],
       [0.8523961 ],
       [0.70531374],
       [0.0650014 ],
       [0.02072064],
       [0.9941981 ],
       [0.14073253],
       [0.65935516],
       [0.1587985 ],
       [0.24299614],
       [0.1453874 ],
       [0.8768521 ],
       [0.36371648],
       [0.06634771],
       [0.4850976 ],
       [0.9521677 ],
       [0.10134061],
       [0.7911903 ],
       [0.9547387 ],
       [0.03345915],
       [0.9992648 ],
       [0.9941981 ],
       [0.09622858],
       [0.99969995],
       [0.03943072],
       [0.05848593],
       [0.53329563],
       [0.30535364],
       [0.28034413],
       [0.9986646 ],
       [0.15236005],
       [0.9605621 ],
       [0.98229915],
       [0.312

In [63]:
# Convert model prediction probabilites to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
       0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1

In [64]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 80.5774278215223,
 'precision': 0.8053042657607273,
 'recall': 0.8057742782152231,
 'f1': 0.805398496755981}

In [65]:
baseline_results

{'accuracy': 79.79002624671917,
 'precision': 0.805269198642199,
 'recall': 0.7979002624671916,
 'f1': 0.7929822281460398}

In [70]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([ True,  True,  True,  True])